In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

from config import gkey

In [61]:
cities_df = pd.read_csv('resources/city_results.csv')

In [62]:
cities_df = cities_df.loc[(cities_df['Temperature (F)'] >= 70) \
                          & (cities_df['Temperature (F)'] <= 80) \
                          & (cities_df['Wind Speed (mph)'] <= 80) \
                          & (cities_df['Cloudiness (%)'] == 0)]

In [75]:
cities_df['Hotel Lat'] = ""
cities_df['Hotel Lng'] = ""
cities_df['Hotel Name'] = ""
cities_df = cities_df.reset_index(drop=True)
cities_df

City Country  Latitude  Longitude  Temperature (F)  \
0                Biloela      AU    -24.42     150.50            76.73   
1               Cabedelo      BR     -6.98     -34.83            78.80   
2                  Arlit      NE     18.74       7.39            78.13   
3                Pahrump      US     36.21    -115.98            72.63   
4                  Bilma      NE     18.69      12.92            73.49   
5                Sambava      MG    -14.27      50.17            70.23   
6                 Rambha      IN     19.52      85.10            75.63   
7                  Erzin      TR     36.96      36.20            71.01   
8           Saint-Joseph      RE    -21.37      55.62            71.60   
9             Takhatgarh      IN     25.33      73.00            70.02   
10           Chandrakona      IN     22.73      87.52            72.59   
11  Makkah al Mukarramah      SA     21.43      39.83            77.27   
12                Calama      CL    -22.47     -68.93            73.40   
13              Guskhara      IN     23.50      87.75            70.83   
14               Pitimbu      BR     -7.47     -34.81            78.80   
15                 Conde      BR     -7.26     -34.91            78.80   
16            Kiryat Gat      IL     31.61      34.76            73.47   
17              Tessalit      ML     20.20       1.01            78.31   
18                Nsanje      MW    -16.92      35.26            73.63   
19               Salalah      OM     17.02      54.09            75.20   
20                   Sur      OM     22.57      59.53            74.93   
21              Shirgaon      IN     19.70      72.70            79.54   
22                 Kidal      ML     18.44       1.41            76.87   
23                Safaga      EG     26.73      33.94            77.00   
24             Abu Dhabi      AE     24.47      54.37            77.23   
25                  Doha      QA     25.29      51.53            78.87   

    Humidity (%)  Cloudiness (%)  Wind Speed (mph) Hotel Name Hotel Lat  \
0             59               0              2.21                        
1             83               0              6.93                        
2             17               0             10.16                        
3             18               0             15.01                        
4             21               0              7.47                        
5             83               0              9.24                        
6             81               0              3.96                        
7             49               0              7.00                        
8             68               0             10.29                        
9             51               0              1.34                        
10            62               0              3.85                        
11            70               0              2.21                        
12            15               0             20.80                        
13            57               0              3.96                        
14            83               0              6.93                        
15            83               0              6.93                        
16            83               0              2.24                        
17            16               0              9.04                        
18            75               0              3.78                        
19            60               0              5.82                        
20            53               0              9.01                        
21            58               0              8.32                        
22            16               0              8.59                        
23            47               0             11.41                        
24            74               0              3.36                        
25            78               0              2.24                     

In [72]:
def get_hotel_from_latlng(lat, lng):
    params = {
        "location": f"{lat},{lng}",
        "rankby": "distance",
        "keyword": "hotel",
        "types": "lodging",
        "key": gkey
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    return requests.get(url, params=params)

In [80]:
for index, row in cities_df.iterrows():
    res = get_hotel_from_latlng(row['Latitude'],row['Longitude']).json()
    try:
        hotel = res["results"][0]
        cities_df.loc[index, 'Hotel Name'] = hotel["name"]
        cities_df.loc[index, 'Hotel Lat'] = hotel["geometry"]["location"]["lat"]
        cities_df.loc[index, 'Hotel Lng'] = hotel["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        cities_df.loc[index, 'Hotel Name'] = np.nan
        cities_df.loc[index, 'Hotel Lat'] = np.nan
        cities_df.loc[index, 'Hotel Lng'] = np.nan

In [84]:
cities_df = cities_df.dropna().reset_index(drop=True)
cities_df.head()

City Country  Latitude  Longitude  Temperature (F)  Humidity (%)  \
0   Biloela      AU    -24.42     150.50            76.73            59   
1  Cabedelo      BR     -6.98     -34.83            78.80            83   
2     Arlit      NE     18.74       7.39            78.13            17   
3   Pahrump      US     36.21    -115.98            72.63            18   
4   Sambava      MG    -14.27      50.17            70.23            83   

   Cloudiness (%)  Wind Speed (mph)            Hotel Name Hotel Lat Hotel Lng  
0               0              2.21  Biloela Caravan Park  -24.4084   150.496  
1               0              6.93     Lua Mansa Pousada  -6.98145   -34.829  
2               0             10.16  Hôtel Telwa Bungalow    18.739   7.38945  
3               0             15.01   Preferred RV Resort   36.2111  -115.982  
4               0              9.24      Hôtel Las Palmas  -14.2619   50.1653